We start with some necessary imports from the pacakge.
To perform the calculations we need:  
a) a DataFrame instance from a row-oriented dataset  
b) a Profile instance from a json/dictionary  
c) a FrameworkScoreValue instance with the properties and scoring profile  


In [1]:
# some necessary imports
import pumas
from pumas.dataframes.dataframe import DataFrame
from pumas.scoring_profile.models import Profile
from pumas.frameworks.frameworks import FrameworkScoreValue

print(pumas.__version__)

0.0.0


In [2]:
data = [
    {"uid": "A", "quality": 1.0, "cost": 30.0, "efficiency": 0.7},
    {"uid": "B", "quality": 5.0, "cost": 40.0, "efficiency": 0.2},
    {"uid": "C", "quality": 3.0, "cost": 10.0, "efficiency": 0.9},
    {"uid": "D", "quality": 6.0, "cost": 60.0, "efficiency": 0.4},
    {"uid": "E", "quality": 2.0, "cost": 80.0, "efficiency": 0.5},
]
properties_dataframe = DataFrame(row_data=data)

We build a ScoringProfile instance from a json/dictionary

In [3]:
scoring_profile_dict = {
    "objectives": [
        {
            "name": "quality",
            "desirability_function": {
                "name": "sigmoid",
                "parameters": {
                    "low": 1.0,
                    "high": 10.0,
                    "k": 0.1,
                    "shift": 0.0,
                    "base": 10.0,
                },
            },
            "weight": 1.0,
            "value_type": "float",
            "kind": "numerical",
        },
        {
            "name": "efficiency",
            "desirability_function": {
                "name": "step",
                "parameters": {
                    "coordinates": [(0.0, 0.0), (0.5, 0.0), (1.0, 0.0)],
                },
            },
            "weight": 2.0,
            "value_type": "float",
            "kind": "numerical",
        },
        {
            "name": "cost",
            "desirability_function": {
                "name": "sigmoid",
                "parameters": {
                    "low": 20.0,
                    "high": 80.0,
                    "k": -0.5,
                    "shift": 0.0,
                    "base": 10.0,
                },
            },
            "weight": 3.0,
            "value_type": "float",
            "kind": "numerical",
        },
    ],
    "aggregation_function": {
        "name": "geometric_mean",
        "parameters": {},
    },
}

scoring_profile = Profile(**scoring_profile_dict)

In [4]:
scorer = FrameworkScoreValue(
    properties=properties_dataframe, scoring_profile=scoring_profile
)
scorer.compute()
desirability_dataframe = scorer.desirability_dataframe
aggregated_dataframe = scorer.aggregated_dataframe

In [5]:
desirability_dataframe.column_data

{'quality': [0.2402530733520421,
  0.4680631904645068,
  0.34533538970278355,
  0.5319368095354932,
  0.28998647022350377],
 'efficiency': [0.0, 0.0, 0.0, 0.0, 0.0],
 'cost': [0.9789100228828135,
  0.8720068922233212,
  0.999536056460154,
  0.12799310777667877,
  0.0031523091832602115]}

In [6]:
aggregated_dataframe.column_data

{'aggregated_score': [np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0)]}